# This notebook provides a recipe for creating netcdf files of profiles by looping through the files on in /results2, /results, /opp
- It has been heavily generalised to provide a general recipe for making the profiles, rather than providing explicit functions that do the job.
- I would estimate that it would take about an hour to concatenate a year's worth of data. Consider a multiprocessing.py approach to creating multiple profiles on a queue. 

In [1]:
import xarray as xr
import numpy as np
from dateutil.parser import parse
# import the file path location from the forcing_paths.py script
import importlib.util
spec = importlib.util.spec_from_file_location("forcing_paths.py", "/ocean/abhudia/MEOPAR/analysis-ashutosh/scripts/make-hdf5/forcing_paths.py")
paths = importlib.util.module_from_spec(spec)
spec.loader.exec_module(paths)

#### Get a list of the files to loop through using the functions in the forcing_paths.py script

#### For the timestart, timeend positional arguments, pass a datetime.datetime object natively or simply use parse

#### for SSC profiles, the filetype argument is 'grid_U', 'grid_V', 'grid_W', 'grid_T' for U, W, V T parameters respectively; e.g. for U files, 
```python
paths.salishseacast_paths(timestart = parse('1 Jan 2015'), timeend = parse('3 jan 2015'), path = '/results2/SalishSea/nowcast-green.201806/', filetype = 'grid_U')
```
#### for WaveWatch files, 
```python
paths.ww3_paths(timestart = parse('1 Jan 2015'), timeend = parse('3 jan 2015'), path = '/opp/wwatch3/nowcast/')
```
#### for HRDPS files,
```python
paths.hrdps_paths(timestart = parse('1 Jan 2015'), timeend = parse('3 jan 2015'), path = '/results/forcing/atmospheric/GEM2.5/operational/')
```

## Let's Step through creating time profiles of surface U current

In [21]:
U_paths = paths.salishseacast_paths(parse('1 Jan 2015'), parse('3 jan 2015'), '/results2/SalishSea/nowcast-green.201806/', 'grid_U')

### From analysis-ashutosh/climatology_analysis_notebooks/Pick grid points.ipynb, we have the locations of the three points chosen as: ((y,x) order)

In [5]:
SB=(np.array([256]), np.array([268]))
TP=(np.array([343]), np.array([250]))
SoG=(np.array([474]), np.array([252]))

### Loop through the list of files and create a netcdf file of the profile

In [37]:
first = True
for file_path in U_paths:
    f = xr.open_dataset(file_path).isel(depthu = 0).vozocrtx
    sog_now = f.isel(y = 256, x = 268)
    if first is True:
        sog = sog_now
        first = False
    else:
        sog = xr.concat((sog, sog_now), dim = 'time_counter')
# finally, stitch them together and turn the profile into a netcdf file
current_u = xr.Dataset({'SoG': sog})
current_u.to_netcdf('current_u_sog_example.nc', format = 'NETCDF4',engine = 'netcdf4')

In [2]:
# loading the netcdf file containing the profile
xr.open_dataset('current_u_sog_example.nc')

<xarray.Dataset>
Dimensions:        (time_counter: 72)
Coordinates:
    nav_lat        float32 ...
    nav_lon        float32 ...
    depthu         float32 ...
    time_centered  (time_counter) datetime64[ns] ...
  * time_counter   (time_counter) datetime64[ns] 2015-01-01T00:30:00 ... 2015-01-03T23:30:00
Data variables:
    SoG            (time_counter) float32 ...